In [1]:
%matplotlib inline
import numpy as np
import pandas as pd

info=pd.read_csv("../../reg_info.csv", na_values=np.nan)
mates=pd.read_csv("../../reg_mates.csv", na_values=np.nan)
df = pd.concat([info, mates]).reset_index(drop=True)
print len(df)

ImportError: No module named Ipython.display

In [ ]:
# Eliminamos la columna ID, ya que no es necesaria
if 'ID' in df:
    df.drop('ID', inplace=True, axis=1)

In [12]:
# Borramos los registros donde aparezca su niub 2 veces i en itirenari_doble aparezca como null. Es decir,
# no tenemos en cuenta aquellos que realizan informática y matemáticas a la vez sin hacer la doble titulación.
# Total de registros borrados: 4
# grouped.filter(lambda x: x["niub"].count()==2 and x['itinerari_doble'].isnull())
df = df.groupby("niub").filter(lambda x: x["niub"].count()!=2 or x['itinerari_doble'].notnull())
print len(df)

917


In [13]:
# Escogemos los de doble titulación de mates+info, i los marcamos como que hacen la doble:
doble = df.groupby("niub").filter(lambda x: x['niub'].count()==2)
doble['enseny'] = 'doble'
for i in doble.index:
    df.loc[i] = doble.loc[i]
print len(df)

917


In [14]:
# Borramos aquellos de doble titulación que no son ni de mates ni de info
#altres_doble = df.groupby("itinerari_doble").filter(lambda x: x['itinerari_doble'] != 'G1077' or x['itinerari_doble'] != 'G1042')
altres = df[df['itinerari_doble'] != 'G1077']
altres = altres[altres['itinerari_doble'] != 'G1042']
altres = altres[altres['itinerari_doble'].notnull()]

print "Alumnes de doble titulació de física + doble titulació de ADE:",len(altres)

df.drop(altres.index, inplace=True)

print len(df)

Alumnes de doble titulació de física + doble titulació de ADE: 154
763


In [15]:
# Eliminem els alumnes amb el niub repetit, ja que els de doble titulació apareixen en el reg_mates i reg_info.
df = df.drop_duplicates('niub')
print len(df)

690


In [16]:
# Reseteamos los indices
df.reset_index(drop=True, inplace=True)

In [17]:
# Per últim eliminarem les columnes 'simultaneitat' i 'itinerari_doble'
if 'simultaneitat' in df:
    df.drop('simultaneitat', inplace=True, axis=1)
if 'itinerari_doble' in df:
    df.drop('itinerari_doble', inplace=True, axis=1)
df.head()

,curs,enseny,niub,sexe,data_naixement,pobl_naixem,codi_prov_naix,prov_naix,codi_pais_naix,pais_naix,codi_nacional,nacionalitat,estat_taxa,tipus_beca,becari,any_inici,itinerari,tipus_acces,via_ingres,nota_acces
0,2013,G1077,16484252,H,18-Aug-94,GUAYAQUIL,0,NaN,148,EQUADOR,100,espanyola,2,NaN,NaN,2013,Z0001,1,7,5.45
1,2013,doble,16528514,H,13-Dec-94,TOMARES,41,Sevilla,100,ESPANYA,100,espanyola,2,NaN,NaN,2013,Z0001,1,1,7.78
2,2013,G1077,16454793,H,2-May-90,BARCELONA,8,Barcelona,100,ESPANYA,100,espanyola,2,NaN,NaN,2013,Z0001,1,37,7.60
3,2013,G1077,14930786,H,21-Apr-86,Barcelona,8,Barcelona,100,ESPANYA,100,espanyola,2,NaN,NaN,2010,Z0001,J,27,0.00
4,2013,doble,16274646,H,22-Apr-87,Terrassa,8,Barcelona,100,ESPANYA,100,espanyola,2,NaN,NaN,2012,Z0001,1,28,5.81


In [18]:
# Aquells que no son becaris tenen en el camp: NaN. Tots aquells els possem com que no són becaris
df['becari'] = df['becari'].fillna('N')

In [19]:
#save pandas to csv
df.to_csv('../../registers.csv', index=False)